<a href="https://colab.research.google.com/github/KIISIISHERE/Legendary-Pokemon-Prediction-usiing-ANN-A.C-Project-/blob/main/Legendary_Pokemon_Prediction_using_ANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
import pandas as pd
df = pd.read_csv('/content/pokemon_data.csv')
df.head()

,Unnamed: 0,against_water,attack,capture_rate,classfication,defense,sp_attack,sp_defense,speed,type1,type2,weight_kg,generation,is_legendary
0,0,0.5,49,45,Seed Pokémon,49,65,65,45,grass,poison,6.9,1,0
1,1,0.5,62,45,Seed Pokémon,63,80,80,60,grass,poison,13.0,1,0
2,2,0.5,100,45,Seed Pokémon,123,122,120,80,grass,poison,100.0,1,0
3,5,2.0,104,45,Flame Pokémon,78,159,115,100,fire,flying,90.5,1,0
4,11,1.0,45,45,Butterfly Pokémon,50,90,80,70,bug,flying,32.0,1,0


In [18]:
df.shape
df.info()
df.isnull().sum()

# Convert 'capture_rate' to numeric, coercing errors to NaN
df['capture_rate'] = pd.to_numeric(df['capture_rate'], errors='coerce')

# Fill any NaN values in 'capture_rate' with the median
df['capture_rate'] = df['capture_rate'].fillna(df['capture_rate'].median())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398 entries, 0 to 397
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Unnamed: 0     398 non-null    int64  
 1   against_water  398 non-null    float64
 2   attack         398 non-null    int64  
 3   capture_rate   398 non-null    float64
 4   defense        398 non-null    int64  
 5   sp_attack      398 non-null    int64  
 6   sp_defense     398 non-null    int64  
 7   speed          398 non-null    int64  
 8   weight_kg      398 non-null    float64
 9   generation     398 non-null    int64  
 10  is_legendary   398 non-null    int64  
dtypes: float64(3), int64(8)
memory usage: 34.3 KB


In [11]:
X = df.drop('is_legendary', axis=1)
y = df['is_legendary']

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [20]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

# Re-apply cleaning steps to X_train and X_test directly
# This is a workaround because X_train and X_test were created before df was fully cleaned.

# 1. Convert 'capture_rate' to numeric, coercing errors to NaN
X_train['capture_rate'] = pd.to_numeric(X_train['capture_rate'], errors='coerce')
X_test['capture_rate'] = pd.to_numeric(X_test['capture_rate'], errors='coerce')

# 2. Fill any NaN values in 'capture_rate' with the median from X_train (to avoid data leakage)
median_capture_rate_train = X_train['capture_rate'].median()
X_train['capture_rate'] = X_train['capture_rate'].fillna(median_capture_rate_train)
X_test['capture_rate'] = X_test['capture_rate'].fillna(median_capture_rate_train)

# 3. Drop categorical columns that StandardScaler cannot handle
categorical_cols_to_drop = ['classfication', 'type1', 'type2']
X_train = X_train.drop(columns=[col for col in categorical_cols_to_drop if col in X_train.columns])
X_test = X_test.drop(columns=[col for col in categorical_cols_to_drop if col in X_test.columns])


X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [22]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()

model.add(Dense(16, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [23]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)


In [24]:
history = model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=32,
    validation_split=0.2
)


Epoch 1/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 35ms/step - accuracy: 0.6266 - loss: 0.7040 - val_accuracy: 0.6406 - val_loss: 0.7206
Epoch 2/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.6920 - loss: 0.6947 - val_accuracy: 0.6875 - val_loss: 0.6641
Epoch 3/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7883 - loss: 0.6066 - val_accuracy: 0.7344 - val_loss: 0.6115
Epoch 4/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7971 - loss: 0.6028 - val_accuracy: 0.8125 - val_loss: 0.5658
Epoch 5/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.8263 - loss: 0.5643 - val_accuracy: 0.8281 - val_loss: 0.5256
Epoch 6/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.8496 - loss: 0.5312 - val_accuracy: 0.8438 - val_loss: 0.4894
Epoch 7/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.8456 - loss: 0.4939 - val_accuracy: 0.8750 - val_loss: 0.4581
Epoch 8/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.8641 - loss: 0.4483 - val_accuracy: 0.8750 - val_loss: 0.4293


In [25]:
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Accuracy:", accuracy)


3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.9297 - loss: 0.2016 
Test Accuracy: 0.9375


In [27]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5).astype(int)

print("Accuracy Score:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Accuracy Score: 0.9375

Classification Report:
               precision    recall  f1-score   support

           0       0.96      0.97      0.97        72
           1       0.71      0.62      0.67         8

    accuracy                           0.94        80
   macro avg       0.84      0.80      0.82        80
weighted avg       0.93      0.94      0.94        80


Confusion Matrix:
 [[70  2]
 [ 3  5]]


# ***Loaded Pokémon Dataset***

### ***What did I do?***

* ***I Preprocessed data***

* ***I Built ANN using Keras***

* ***I Trained the model***

* ***I Evaluated performance on test dataset***